In [1]:
import pandas as pd
from utils import home, join, get_files, read_metadata_without_fields, read_metadata_with_fields
import numpy as np

In [2]:
rna_seq = pd.read_excel('pdxe_mol_data.xlsx', sheet_name='RNAseq_fpkm')
cna = pd.read_excel('pdxe_mol_data.xlsx', sheet_name='copy number')

In [80]:
dfci = join(home, 'DFCI-CPDM')
files = [f for f in get_files(dfci) if f.endswith('tsv')]
ps = read_metadata_without_fields(join(home, 'DFCI-CPDM/DFCI-CPDM_metadata-patient_sample.tsv'))
pdx = read_metadata_without_fields(join(home, 'DFCI-CPDM/DFCI-CPDM_metadata-pdx_model.tsv'))
mms = read_metadata_without_fields(join(home, 'DFCI-CPDM/DFCI-CPDM_molecular_metadata-sample.tsv'))
mms_header = read_metadata_with_fields(join(home, 'DFCI-CPDM/DFCI-CPDM_molecular_metadata-sample.tsv')).iloc[0:4,:]

expression_template = read_metadata_with_fields('expression_template-sheet.tsv')
samples = 'NIBR' + rna_seq.columns
matched_samples = rna_seq.columns[[True if s in ps.model_id.to_list() or s == 'NIBRSample' else False for s in samples]]
df = rna_seq[matched_samples]

for sample in df.columns[1:]:
    out_df = pd.DataFrame(columns=expression_template.columns)
    sample_id = 'NIBR' + sample.replace('X', '')
    out_df['rnaseq_fpkm'] = df[sample]
    out_df['symbol'] = df['Sample']
    out_df['sample_id'] = sample_id
    out_df['platform_id'] = 'expression_rna_seq'
    sum_fpkm = out_df['rnaseq_fpkm'].sum()
    out_df['rnaseq_tpm'] = (out_df['rnaseq_fpkm'] * 1e6) / sum_fpkm
    pdx_row = pdx[pdx['model_id'] == 'NIBR' + sample].reset_index(drop=True)
    mms_row = [['NIBR' + sample, sample_id, 'xenograft', pdx_row['passage_number'][0], pdx_row['host_strain_name'][0], pdx_row['host_strain_nomenclature'][0], pdx_row['engraftment_site'][0], '', 'expression_rna_seq']]
    mms = pd.concat([mms, pd.DataFrame(mms_row, columns=mms.columns)])
    out_df.to_csv(f'expression/DFCI-CPDM_expression_NIBR{sample}.tsv', sep='\t', index=False)
mms['Field'] = ''
mms = pd.concat([mms_header, mms])

In [79]:
mms.to_csv(join(home, 'DFCI-CPDM/DFCI-CPDM_molecular_metadata-sample.tsv'), sep='\t', index=False)

In [7]:
cna.iloc[:,1:].max().max()

173.65

In [8]:
cna.iloc[:,1:].min().min()

0.02

In [10]:
cna.iloc[:,1:].min().max()

0.43

In [14]:
cna.max() [cna.max() == 173.65]

X-4132    173.65
dtype: object

In [16]:
cna[['Sample', 'X-4132']]

,Sample,X-4132
0,ArmLevelCNScore,0.47
1,FocalCNScore,0.18
2,A1BG,2.01
3,A1BG-AS1,2.01
4,A1CF,1.42
...,...,...
23849,ZYG11A,1.39
23850,ZYG11B,2.51
23851,ZYX,2.01
23852,ZZEF1,1.39
